In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jobathon-nov-2022/sample_submission.csv
/kaggle/input/jobathon-nov-2022/train.csv
/kaggle/input/jobathon-nov-2022/test.csv


In [2]:
!pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 3.8 MB/s eta 0:00:00


In [3]:
train=pd.read_csv('/kaggle/input/jobathon-nov-2022/train.csv')
print(train.shape)

(94992, 3)


In [4]:
test=pd.read_csv('/kaggle/input/jobathon-nov-2022/test.csv')
print(test.shape)

(26304, 2)


In [5]:
train.head()

,row_id,datetime,energy
0,1,2008-03-01 00:00:00,1259.985563
1,2,2008-03-01 01:00:00,1095.541500
2,3,2008-03-01 02:00:00,1056.247500
3,4,2008-03-01 03:00:00,1034.742000
4,5,2008-03-01 04:00:00,1026.334500


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94992 entries, 0 to 94991
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   row_id    94992 non-null  int64  
 1   datetime  94992 non-null  object 
 2   energy    93092 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.2+ MB


In [7]:
train['energy'].describe()

count    93092.000000
mean      1702.995014
std        369.322103
min        866.388600
25%       1431.541100
50%       1654.465800
75%       1935.993450
max       3374.399600
Name: energy, dtype: float64

In [8]:
train['datetime']=pd.to_datetime(train['datetime'],infer_datetime_format=True)

In [9]:
train['datetime'].min(),train['datetime'].max()

(Timestamp('2008-03-01 00:00:00'), Timestamp('2018-12-31 23:00:00'))

In [10]:
hour_map={0:0,1:0,2:0,3:0,4:0,5:0,
         6:1,7:1,8:1,
         9:2,10:2,11:2,
         12:3,13:3,14:3,15:3,
         16:4,17:4,
         18:5,19:5,20:5,
         21:6,22:6,23:6}

In [11]:
train['year']=train['datetime'].dt.year
train['month']=train['datetime'].dt.month
train['day']=train['datetime'].dt.day
train['hour']=train['datetime'].dt.hour
train['is_weekend']=train['datetime'].dt.dayofweek >4
train['day_part']=train['hour'].replace(hour_map)

In [12]:
train['year'].unique()

array([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018])

In [13]:
test['datetime']=pd.to_datetime(test['datetime'],infer_datetime_format=True)

In [14]:
test['datetime'].min(),test['datetime'].max()

(Timestamp('2019-01-01 00:00:00'), Timestamp('2021-12-31 23:00:00'))

In [15]:
test['year']=test['datetime'].dt.year
test['month']=test['datetime'].dt.month
test['day']=test['datetime'].dt.day
test['hour']=test['datetime'].dt.hour
test['is_weekend']=test['datetime'].dt.dayofweek >4
test['day_part']=test['hour'].replace(hour_map)

In [16]:
targetcol = 'energy'

In [17]:
train.fillna(train[targetcol].mean(),inplace=True)

Split Train into train and validation sets

In [18]:
# import cudf
# from cuml.tsa.arima import ARIMA

In [19]:
import datetime 
train_start = datetime.datetime(year=2008,month=1,day=1,hour=0)
val_start = datetime.datetime(year=2016,month=1,day=1,hour=0)
val_end = datetime.datetime(year=2018,month=12,day=31,hour=23)
# train_start = datetime.datetime(year=2017,month=1,day=1,hour=0)
# val_start = datetime.datetime(year=2018,month=1,day=1,hour=0)
# val_end = datetime.datetime(year=2018,month=12,day=31,hour=23)
hour_mask = (train['hour']>=0)
X_val= train[(train['datetime']>=val_start) & (train['datetime']<=val_end) & hour_mask].copy()
X_train= train[(train['datetime']>=train_start) & (train['datetime']<val_start) & hour_mask].copy()
print(X_train.shape)
print(X_val.shape)
X_val.head()           

(68688, 9)
(26304, 9)


,row_id,datetime,energy,year,month,day,hour,is_weekend,day_part
68688,68689,2016-01-01 00:00:00,2002.2470,2016,1,1,0,False,0
68689,68690,2016-01-01 01:00:00,1843.3870,2016,1,1,1,False,0
68690,68691,2016-01-01 02:00:00,1735.1178,2016,1,1,2,False,0
68691,68692,2016-01-01 03:00:00,1666.1970,2016,1,1,3,False,0
68692,68693,2016-01-01 04:00:00,1630.3924,2016,1,1,4,False,0


In [20]:
val_target = X_val[targetcol]
print(X_val[targetcol].isnull().sum())

0


In [21]:
# val_target.fillna(X_train[targetcol].mean(),inplace=True)

In [22]:
from sklearn.metrics import mean_squared_error

In [23]:
#compute baseline error by predicting train energy mean as the energy for all time
val_preds_baseline = np.full(len(X_val),train[targetcol].mean())

In [24]:
# print(X_train[targetcol].isnull().sum())
# X_train.fillna(X_train[targetcol].mean(),inplace=True)
# print(X_train[targetcol].isnull().sum())

In [25]:
print(val_target[0:5])

68688    2002.2470
68689    1843.3870
68690    1735.1178
68691    1666.1970
68692    1630.3924
Name: energy, dtype: float64


In [26]:
#compute error score on baseline predictions
val_score = mean_squared_error(val_target,val_preds_baseline,squared=False)
print('valid score:',val_score)

valid score: 427.29210032008933


UCM

In [27]:
X_val.shape
X_train.shape

(68688, 9)

In [28]:
import statsmodels.api as sm

In [29]:
# X_train= X_train.set_index(X_train.datetime).sort_index()
# X_val= X_val.set_index(X_val.datetime).sort_index()

In [30]:
%%time
#splitting time series to train and test subsets
y_train = X_train[targetcol].copy()
y_test = X_val[targetcol].copy()

exog_train = X_train[['is_weekend','month','hour']].astype('int')
exog_test = X_val[['is_weekend','month','hour']].astype('int')

#Unobserved Components model definition
model_UC1 = sm.tsa.UnobservedComponents(y_train,
                                        autoregressive=2,
                                        level='lldtrend',
                                        exog=exog_train,
                                        cycle=False,
                                        irregular=False,
                                        stochastic_level = False,
                                        stochastic_trend = False,
                                        stochastic_freq_seasonal = [False,False,True],
#                                         seasonal_period =24,
                                        freq_seasonal=[{'period': 24, 'harmonics': 1},
                                                       {'period': 168, 'harmonics': 1},
#                                                        {'period': 5040, 'harmonics': 1},
                                                       {'period': 8766, 'harmonics': 2}
                                                      ]
                                       )
#fitting model to train data
model_UC1res = model_UC1.fit()

#printing statsmodels summary for model
print(model_UC1res.summary())

print("")

#calculating mean absolute error and root mean squared error for in-sample prediction of model
print(f"In-sample mean absolute error (MAE): {'%.0f' % model_UC1res.mae}, In-sample root mean squared error (RMSE): {'%.0f' % np.sqrt(model_UC1res.mse)}")

#model forecast

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            9     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.74947D+00    |proj g|=  3.54122D-03


 This problem is unconstrained.



At iterate    5    f=  6.71677D+00    |proj g|=  2.18922D-02

At iterate   10    f=  5.86081D+00    |proj g|=  4.27136D-03

At iterate   15    f=  5.81348D+00    |proj g|=  9.86097D-04

At iterate   20    f=  5.81284D+00    |proj g|=  5.12348D-04

At iterate   25    f=  5.81119D+00    |proj g|=  4.17727D-03

At iterate   30    f=  5.80957D+00    |proj g|=  1.07680D-03

At iterate   35    f=  5.80945D+00    |proj g|=  4.03421D-05

At iterate   40    f=  5.80922D+00    |proj g|=  4.28360D-04

At iterate   45    f=  5.80916D+00    |proj g|=  3.55446D-05

At iterate   50    f=  5.80912D+00    |proj g|=  1.21617D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tn

/opt/conda/lib/python3.7/site-packages/statsmodels/base/model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


                                 Unobserved Components Results                                 
Dep. Variable:                                  energy   No. Observations:                68688
Model:                local linear deterministic trend   Log Likelihood             -399016.808
                                + freq_seasonal(24(1))   AIC                         798051.616
                               + freq_seasonal(168(1))   BIC                         798133.851
                   + stochastic freq_seasonal(8766(2))   HQIC                        798077.001
                                               + AR(2)                                         
Date:                                 Sat, 19 Nov 2022                                         
Time:                                         13:58:22                                         
Sample:                                              0                                         
                                        

In [31]:
forecast_UC1 = model_UC1res.forecast(steps=26304,exog=exog_test)
#calculating root mean squared error
RMSE_UC1 = mean_squared_error(y_test,forecast_UC1,squared=False)
print('valid score:',RMSE_UC1)

valid score: 279.16511925488896


In [32]:
pd.Series(forecast_UC1.describe())

count    26304.000000
mean      2106.579278
std        193.114917
min       1606.235923
25%       1968.989750
50%       2106.503829
75%       2244.389964
max       2626.933114
Name: predicted_mean, dtype: float64

In [33]:
# from sklearn.preprocessing import MinMaxScaler
# import tensorflow as tf
# import os

In [34]:
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.Series(diff)

In [35]:
import matplotlib.pyplot as plt

UC for Test Prediction

In [36]:
%%time
#splitting time series to train and test subsets
y_train_full = train[targetcol].copy()

exog_train_full = train[['is_weekend','month','hour']].astype('int')
exog_test_full = test[['is_weekend','month','hour']].astype('int')

#Unobserved Components model definition
model_UC_full = sm.tsa.UnobservedComponents(y_train_full,
                                        autoregressive=2,
                                        level='lldtrend',
                                        exog=exog_train_full,
                                        cycle=False,
                                        irregular=False,
                                        stochastic_level = False,
                                        stochastic_trend = False,
                                        stochastic_freq_seasonal = [False,False,True],
#                                         seasonal_period =24,
                                        freq_seasonal=[{'period': 24, 'harmonics': 1},
                                                       {'period': 168, 'harmonics': 1},
#                                                        {'period': 5040, 'harmonics': 1},
                                                       {'period': 8766, 'harmonics': 2}
                                                      ]
                                       )
#fitting model to train data
model_UC_full_res = model_UC_full.fit()

#printing statsmodels summary for model
print(model_UC_full_res.summary())

print("")

#calculating mean absolute error and root mean squared error for in-sample prediction of model
print(f"In-sample mean absolute error (MAE): {'%.0f' % model_UC_full_res.mae}, In-sample root mean squared error (RMSE): {'%.0f' % np.sqrt(model_UC_full_res.mse)}")

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            9     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.87063D+00    |proj g|=  3.38148D-03

At iterate    5    f=  6.69184D+00    |proj g|=  2.28698D-02

At iterate   10    f=  5.90283D+00    |proj g|=  2.71825D-03

At iterate   15    f=  5.88951D+00    |proj g|=  1.81722D-04

At iterate   20    f=  5.88747D+00    |proj g|=  1.25732D-03

At iterate   25    f=  5.88644D+00    |proj g|=  1.07707D-03

At iterate   30    f=  5.88610D+00    |proj g|=  4.26233D-04

At iterate   35    f=  5.88609D+00    |proj g|=  6.02562D-05

At iterate   40    f=  5.88607D+00    |proj g|=  3.10294D-04

At iterate   45    f=  5.88594D+00    |proj g|=  1.61634D-04


/opt/conda/lib/python3.7/site-packages/statsmodels/base/model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)



At iterate   50    f=  5.88591D+00    |proj g|=  5.06279D-06

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    9     50     66      1     0     0   5.063D-06   5.886D+00
  F =   5.8859056646091723     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
                                 Unobserved Components Results                                 
Dep. Variable:                                  energy   No. Observations:                94992
Model:                local linear deterministic trend   Log Likelihood             -559113.951
                                + freq_seasonal(24(1))   AIC             

In [37]:
#Predict Test Set
forecast_UC_test = model_UC_full_res.forecast(steps=26304,exog=exog_test_full)

In [38]:
print(len(test),len(forecast_UC_test))

26304 26304


In [39]:
forecast_UC_test.index

RangeIndex(start=94992, stop=121296, step=1)

In [40]:
pd.Series(forecast_UC_test).describe()

count    26304.000000
mean      2245.786163
std        206.741366
min       1706.726121
25%       2098.440395
50%       2244.774674
75%       2392.296883
max       2814.446614
Name: predicted_mean, dtype: float64

In [41]:
subm = pd.DataFrame()
subm['row_id'] = test['row_id']
subm[targetcol]=forecast_UC_test.values
subm.to_csv('submission.csv',index=False)

In [42]:
#check saved submission data
pd.read_csv('submission.csv').head()

,row_id,energy
0,94993,2038.440855
1,94994,2006.741063
2,94995,1994.880913
3,94996,1987.600845
4,94997,1992.652133


In [43]:
# import pmdarima as pm